In [1]:
## Bring in needed mods
import pandas as pd, numpy as np, glob, sys, os, seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.insert(0, '/home/croth/QTL-functions/SCRIPTS/')
import qtlfunctions as qtlfun

In [2]:
## Set chromosomes
chromns = np.arange(1,15)

In [3]:
## Gather allelic depth
ads = []

## Iterate thru chorms
for i,c in enumerate(chromns):
    
    ## Gather paths
    info_path = glob.glob('../GENOTYPE/INFO/*.%s.B3502.progeny_info_cols.csv.gz'%c)[0]
    dept_path = glob.glob('../GENOTYPE/AF/*.%s.B3502.progeny_allele_ratios.csv.gz'%c)[0]
    
    ## Load info and depth
    info = pd.read_csv(info_path,index_col=0)
    dp = pd.read_csv(dept_path,index_col=0)
    
    ## Concat dataframes
    depth = pd.concat([info,dp],axis=1)
    
    ## Append
    ads.append(depth)
    
## Concat rows
ads = pd.concat(ads,axis=0)

## View head
ads.head()

,Seqid,Pos,Qual,Nalleles,Alleles,Maxlen,Minlen,Type,Callrate,MAF,...,A15,A16,A17,A20,A23,A30,A31,A33,A32,A34
1,AE017341.1,21,0.000000e+00,1,TTCTC.CTCTG,5,5,MNP,1.00000,0.989130,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25,AE017341.1,317,3.092230e-09,2,CAGT.TAT.TAGT,4,3,del,1.00000,0.989130,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
26,AE017341.1,331,3.177140e+02,1,ACAATCCG.ACAAGCCA,8,8,MNP,1.00000,0.989130,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
31,AE017341.1,393,3.032550e+03,1,A.G,1,1,SNP,0.98913,0.978261,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
34,AE017341.1,435,2.502520e+03,1,G.C,1,1,SNP,0.98913,0.978261,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
## Gather depths
dps = []

## Loop thru chroms
for i,c in enumerate(chromns):
    
    ## Get paths
    info_path = glob.glob('../GENOTYPE/INFO/*.%s.B3502.progeny_info_cols.csv.gz'%c)[0]
    dept_path = glob.glob('../GENOTYPE/DP/*.%s.B3502.progeny_depths.csv.gz'%c)[0]
    
    ## Load df
    info = pd.read_csv(info_path,index_col=0)
    dp = np.log2(pd.read_csv(dept_path,index_col=0)+1)
    
    ## Concat columns
    depth = pd.concat([info,dp],axis=1)
    
    ## Append dataframe
    dps.append(depth)
    
## Concat rows
dps = pd.concat(dps,axis=0)

## View head
dps.head()

,Seqid,Pos,Qual,Nalleles,Alleles,Maxlen,Minlen,Type,Callrate,MAF,...,A15,A16,A17,A20,A23,A30,A31,A33,A32,A34
1,AE017341.1,21,0.000000e+00,1,TTCTC.CTCTG,5,5,MNP,1.00000,0.989130,...,6.392317,6.266787,5.832890,6.686501,6.894818,4.906891,5.832890,5.700440,5.754888,5.392317
25,AE017341.1,317,3.092230e-09,2,CAGT.TAT.TAGT,4,3,del,1.00000,0.989130,...,7.357552,7.087463,6.507795,7.357552,7.643856,6.392317,6.228819,6.523562,6.954196,5.832890
26,AE017341.1,331,3.177140e+02,1,ACAATCCG.ACAAGCCA,8,8,MNP,1.00000,0.989130,...,7.238405,6.918863,6.539159,7.303781,7.515700,6.209453,6.169925,6.507795,6.942515,5.832890
31,AE017341.1,393,3.032550e+03,1,A.G,1,1,SNP,0.98913,0.978261,...,7.285402,6.965784,6.285402,7.257388,7.546894,6.409391,6.321928,6.392317,6.820179,6.129283
34,AE017341.1,435,2.502520e+03,1,G.C,1,1,SNP,0.98913,0.978261,...,7.294621,7.076816,6.643856,7.209453,7.562242,6.339850,6.442943,6.459432,6.832890,6.189825


In [5]:
## Gather sample names and print length
samples = np.unique(dp.columns)
print(len(samples))

## Gather progeny
progeny = sorted([s for s in samples if len(s)<=4])

## Gather ancestors
ancestors = sorted([s for s in samples if len(s)>4])

## Check work
assert len(progeny) + len(ancestors) == len(samples)

## Gather B3502 stocks
b3502 = [s for s in ancestors if s.split("_")[0]=='B3502']

## parents and progenitor strains
progenitors = [s for s in ancestors if s not in b3502]

92


In [7]:
## For each sample, plot the depth and allelic depth ratio
for sample in samples:

    ## Call figure
    fig,ax = plt.subplots(2,1,figsize=(12,5),sharex=True)
    fig.set_facecolor('w')

    ## Set axis and title
    plt.sca(ax[0])
    plt.title(sample)

    ## plot log2 depth
    qtlfun.manhattan(dps,pval=sample,chrom='Seqid',xlabel='',
                 ylabel='log$_2$ (Depth)',ylims=(0,12),ax=ax[0]);

    ## plot allelic depth ratio
    qtlfun.manhattan(ads,pval=sample,chrom='Seqid',chromlabels=chromns,
                 ylabel='Allelic Depth',ylims=(0,1),ax=ax[1]);
    
    ## Save figure and close figure
    plt.savefig('../FIGURES/DEPTHPLOTS/%s.png'%sample,
                dpi=100,bbox_inches='tight')
    plt.close()